In [1]:
import os
def log(string):
    os.system('echo '+string)

log("START")

START


In [2]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import notebook
import os
import time
import requests
from PIL import Image
from io import BytesIO
import tensorflow as tf
from keras import layers, metrics, optimizers
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import gc
import logging

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class Product:

 def __init__(self, link, title, imageLinks):
   self.link = link
   self.title = title
   self.imageLinks = imageLinks

In [4]:
db = pickle.load(open('/kaggle/input/tshirt-database/tshirt_2_database.pkl', 'rb'))
log(str(len(db)))

10000


In [5]:
inputShape = (224, 224, 3)

def loadImage(url,shape):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).resize((shape[0],shape[1])).convert('RGB')

def preprocess_image(imgUrl):
    img = loadImage(imgUrl,(inputShape[0], inputShape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)[0]
    return preprocessed_img

In [6]:
model = ResNet50(
    weights="imagenet", input_shape=inputShape, include_top=False
)

94765736/94765736 [==============================] - 0s 0us/step


In [7]:
feature_list = []
for i in notebook.tqdm(range(len(db))):
    try:
        preprocessed_img = np.array([preprocess_image(db[i].imageLinks[0])])
        features = np.array(model(preprocessed_img)[0]).reshape(1,-1)[0]
        print(np.shape(features))
        feature_list.append((i,features))
    except:
        print("SKIP "+str(i))
log("FEATURE EXTRACTION END")
pickle.dump(feature_list, open('imagenet_tshirt3_features.pkl', 'wb'))
log("FEATURES SAVED")

  0%|          | 0/10000 [00:00<?, ?it/s]

(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)
(100352,)


In [8]:
feature_list = pickle.load(open('imagenet_tshirt3_features.pkl', 'rb'))

In [9]:
features = []
for pair in feature_list:
    features.append(pair[1])

In [10]:
neighbors = NearestNeighbors(n_neighbors=10, algorithm='brute',metric='cosine').fit(features)

In [11]:
def display_images(images, titles=None, figsize=(15, 3)):
   """Displays a list of images in a single row using matplotlib.

   Args:
       images: A list of images (as NumPy arrays or PIL images).
       titles: A list of titles for each image (optional).
       figsize: The desired figure size (width, height).
   """

   num_images = len(images)

   # Create a figure with subplots in a single row
   fig, axes = plt.subplots(1, num_images, figsize=figsize)

   # Show each image with its corresponding title (if provided)
   for i, image in enumerate(images):
       ax = axes[i] if num_images > 1 else axes
       ax.imshow(image)
       ax.axis('off')  # Remove axis ticks and labels
       if titles and i < len(titles):
           ax.set_title(titles[i])

   # Adjust layout for tight spacing
   plt.tight_layout()
   plt.show()

In [12]:
testUrl = "https://www.montecarlo.in/cdn/shop/products/223062048-1-38_1.jpg?v=1701239841&width=1080"
features = np.array(embedding(np.array([preprocess_image(testUrl)]))).reshape(1,-1)
distances, indices = neighbors.kneighbors(features)

NameError: name 'embedding' is not defined

In [ ]:
imgs = []
titles = []
for (index, distance) in zip(indices[0], distances[0]):
    response = requests.get(db[feature_list[index][0]].imageLinks[0])
    img = Image.open(BytesIO(response.content))
    imgs.append(img)
    titles.append(float("{:.2f}".format(distance)))
display_images(imgs,titles)